# 한국은행 경제통계시스템 Open API
+ 작성: 임경호
+ 한국은행 경제통계 Open API https://ecos.bok.or.kr/api/#/

### 서비스 통계 목록

In [38]:
import requests
import json
import pandas as pd

api_key = 'T4GS4G5ZJSQI9HOTYV64'

service_name = 'StatisticTableList'        # 서비스 통계 목록
req_type = 'json'
req_start, req_end = '1', '1000'
stat_code = ''                      # 통계표코드. 지정하지 않으면 전체 가져옴

url = f'http://ecos.bok.or.kr/api/{service_name}/{api_key}/{req_type}/kr/{req_start}/{req_end}'

response = requests.get(url)
if response.status_code != 200:     # URL GET '200 정상'
    print("URL GET Error")
    
jo = json.loads(response.text)
df_result = pd.json_normalize(jo[service_name]['row'])

In [39]:
df_result.shape

(820, 6)

+ 결과값
  - P_STAT_CODE	: 상위통계표코드
  - STAT_CODE : 통계표코드	
  - STAT_NAME : 통계명	
  - CYCLE : 주기(년, 분기, 월)	
  - SRCH_YN	: 검색가능여부
  - ORG_NAME : 출처

In [41]:
df_result.head(10)

,P_STAT_CODE,STAT_CODE,STAT_NAME,CYCLE,SRCH_YN,ORG_NAME
0,*,0000000001,1. 통화/금융,None,N,None
1,0000000001,0000000002,1.1. 통화/유동성,None,N,None
2,0000000002,0000000003,1.1.1. 본원통화,None,N,None
3,0000000003,0000000004,1.1.1.1. 본원통화 구성내역,None,N,None
4,0000000004,102Y004,"1.1.1.1.1. 본원통화 구성내역(평잔, 계절조정계열)",M,Y,None
5,0000000004,102Y002,"1.1.1.1.2. 본원통화 구성내역(평잔, 원계열)",M,Y,None
6,0000000004,102Y003,"1.1.1.1.3. 본원통화 구성내역(말잔, 계절조정계열)",M,Y,None
7,0000000004,102Y001,"1.1.1.1.4. 본원통화 구성내역(말잔, 원계열)",M,Y,한국은행
8,0000000002,0000000005,1.1.2. M1(협의통화),None,N,None
9,0000000005,0000000006,1.1.2.1. M1(협의통화) 상품별 구성내역,None,N,None


In [42]:
df_result.to_excel('result/'+service_name+'.xlsx')

### 100대 통계지표

In [30]:
import requests
import json
import pandas as pd

api_key = 'T4GS4G5ZJSQI9HOTYV64'

service_name = 'KeyStatisticList'        # 100대 통계지표
req_type = 'json'
req_start, req_end = '1', '100'

url = f'http://ecos.bok.or.kr/api/{service_name}/{api_key}/{req_type}/kr/{req_start}/{req_end}'

response = requests.get(url)
if response.status_code != 200:     # URL GET '200 정상'
    print("URL GET Error")
    
jo = json.loads(response.text)
df_result = pd.json_normalize(jo[service_name]['row'])

In [31]:
df_result.shape

(100, 5)

+ 결과
  - CLASS_NAME : 통계그룹명
  - KEYSTAT_NAME : 통계명
  - DATA_VALUE : 값
  - CYCLE : 통계의 최근수록 시점
  - UNIT_NAME : 단위

In [32]:
df_result

,CLASS_NAME,KEYSTAT_NAME,DATA_VALUE,CYCLE,UNIT_NAME
0,시장금리,한국은행 기준금리,3.5,20230703,%
1,시장금리,콜금리(익일물),3.454,20230704,%
2,시장금리,KORIBOR(3개월),3.79,20230705,%
3,시장금리,CD수익률(91일),3.74,20230705,%
4,시장금리,통안증권수익률(364일),3.586,20230705,%
...,...,...,...,...,...
95,부동산 가격,주택매매가격지수,95.686189,202305,2021.6=100
96,부동산 가격,주택전세가격지수,91.569451,202305,2021.6=100
97,부동산 가격,지가변동률(전기대비),0.043,202305,%
98,원자재,"국제유가(Dubai, 현물)",76.05,202306,달러


In [34]:
df_result.to_excel('result/'+service_name+'.xlsx')

### 통계조회 (모듈화)

In [95]:
def bok_statistic_search(statcode, freq, period_start, period_end, item, req_start, req_end):
    api_key = 'T4GS4G5ZJSQI9HOTYV64'
    service_name = 'StatisticSearch'
    req_type = 'json'
    url = f'http://ecos.bok.or.kr/api/{service_name}/{api_key}/{req_type}/kr/{req_start}/{req_end}/{statcode}/{freq}/{period_start}/{period_end}/{item}'
    response = requests.get(url)
    if response.status_code != 200:     # URL GET '200 정상'
        return None
    jo = json.loads(response.text)
    result = pd.json_normalize(jo[service_name]['row'])
    return result

In [96]:
# 통계조회: 주요국 통화의 대원화환율 - 원/미국달러(매매기준율)
df_result1 = bok_statistic_search(statcode='731Y001', freq='D', period_start='20230101', period_end='20230630', item='0000001' , req_start='1', req_end='200')
# 통계조회: 소비자물가지수
df_result2 = bok_statistic_search(statcode='901Y009', freq='M', period_start='200001', period_end='202005', item='0' , req_start='1', req_end='10000')

In [97]:
df_result1.head(2)

,STAT_CODE,STAT_NAME,ITEM_CODE1,ITEM_NAME1,ITEM_CODE2,ITEM_NAME2,ITEM_CODE3,ITEM_NAME3,ITEM_CODE4,ITEM_NAME4,UNIT_NAME,TIME,DATA_VALUE
0,731Y001,3.1.1.1. 주요국 통화의 대원화환율,0000001,원/미국달러(매매기준율),None,None,None,None,None,None,원,20230102,1267.3
1,731Y001,3.1.1.1. 주요국 통화의 대원화환율,0000001,원/미국달러(매매기준율),None,None,None,None,None,None,원,20230103,1268.9


In [98]:
df_result2.head(2)

,STAT_CODE,STAT_NAME,ITEM_CODE1,ITEM_NAME1,ITEM_CODE2,ITEM_NAME2,ITEM_CODE3,ITEM_NAME3,ITEM_CODE4,ITEM_NAME4,UNIT_NAME,TIME,DATA_VALUE
0,901Y009,4.2.1. 소비자물가지수,0,총지수,None,None,None,None,None,None,2020=100,200001,62.456
1,901Y009,4.2.1. 소비자물가지수,0,총지수,None,None,None,None,None,None,2020=100,200002,62.647


### 통계조회: 주요국 통화의 대원화환율

In [83]:
import requests
import json
import pandas as pd

api_key = 'T4GS4G5ZJSQI9HOTYV64'

service_name = 'StatisticSearch'
statcode = '731Y001'
req_type = 'json'
freq, start, end = 'D', '20230101', '20230630'
# item = ''     # 지정하지 않으면 전체 대상
item = '0000001'    # 원/미국달러(매매기준율)

url = f'http://ecos.bok.or.kr/api/{service_name}/{api_key}/{req_type}/kr/1/10000/{statcode}/{freq}/{start}/{end}/{item}'

response = requests.get(url)
if response.status_code != 200:     # URL GET '200 정상'
    print("URL GET Error")
    
jo = json.loads(response.text)
# print(jo['RESULT']['CODE'])
df_result = pd.json_normalize(jo[service_name]['row'])

In [84]:
df_result.shape

(123, 13)

In [85]:
df_result.head()

,STAT_CODE,STAT_NAME,ITEM_CODE1,ITEM_NAME1,ITEM_CODE2,ITEM_NAME2,ITEM_CODE3,ITEM_NAME3,ITEM_CODE4,ITEM_NAME4,UNIT_NAME,TIME,DATA_VALUE
0,731Y001,3.1.1.1. 주요국 통화의 대원화환율,0000001,원/미국달러(매매기준율),None,None,None,None,None,None,원,20230102,1267.3
1,731Y001,3.1.1.1. 주요국 통화의 대원화환율,0000001,원/미국달러(매매기준율),None,None,None,None,None,None,원,20230103,1268.9
2,731Y001,3.1.1.1. 주요국 통화의 대원화환율,0000001,원/미국달러(매매기준율),None,None,None,None,None,None,원,20230104,1272.1
3,731Y001,3.1.1.1. 주요국 통화의 대원화환율,0000001,원/미국달러(매매기준율),None,None,None,None,None,None,원,20230105,1274.7
4,731Y001,3.1.1.1. 주요국 통화의 대원화환율,0000001,원/미국달러(매매기준율),None,None,None,None,None,None,원,20230106,1270.3


In [86]:
df_result.to_excel('result/'+service_name+'_'+statcode+'.xlsx')

### 통계조회: 소비자물가지수

In [81]:
import requests
import json
import pandas as pd

api_key = 'T4GS4G5ZJSQI9HOTYV64'

service_name = 'StatisticSearch'
statcode = '901Y009'        # 소비자물가지수
req_type = 'json'
freq, start, end, item = 'M', '200001', '202005', '0'

url = f'http://ecos.bok.or.kr/api/{service_name}/{api_key}/{req_type}/kr/1/10000/{statcode}/{freq}/{start}/{end}/{item}'

response = requests.get(url)
if response.status_code != 200:     # URL GET '200 정상'
    print("URL GET Error")
    
jo = json.loads(response.text)
df_result = pd.json_normalize(jo[service_name]['row'])

In [82]:
df_result.shape

(245, 13)

In [56]:
df_result

,STAT_CODE,STAT_NAME,ITEM_CODE1,ITEM_NAME1,ITEM_CODE2,ITEM_NAME2,ITEM_CODE3,ITEM_NAME3,ITEM_CODE4,ITEM_NAME4,UNIT_NAME,TIME,DATA_VALUE
0,901Y009,4.2.1. 소비자물가지수,0,총지수,None,None,None,None,None,None,2020=100,200001,62.456
1,901Y009,4.2.1. 소비자물가지수,0,총지수,None,None,None,None,None,None,2020=100,200002,62.647
2,901Y009,4.2.1. 소비자물가지수,0,총지수,None,None,None,None,None,None,2020=100,200003,62.962
3,901Y009,4.2.1. 소비자물가지수,0,총지수,None,None,None,None,None,None,2020=100,200004,62.709
4,901Y009,4.2.1. 소비자물가지수,0,총지수,None,None,None,None,None,None,2020=100,200005,62.393
...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,901Y009,4.2.1. 소비자물가지수,0,총지수,None,None,None,None,None,None,2020=100,202001,100.09
241,901Y009,4.2.1. 소비자물가지수,0,총지수,None,None,None,None,None,None,2020=100,202002,100.16
242,901Y009,4.2.1. 소비자물가지수,0,총지수,None,None,None,None,None,None,2020=100,202003,99.94
243,901Y009,4.2.1. 소비자물가지수,0,총지수,None,None,None,None,None,None,2020=100,202004,99.5


In [57]:
df_result.to_excel('result/'+service_name+'_'+statcode+'.xlsx')